In [1]:
import warnings
import os
from pathlib import Path 
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
import anndata
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
import decoupler as dc
import itertools
from scipy import sparse
#import lib.data_helpers as dh

#import lib.scanpy_helpers as sh
#import lib.pl.util as pu

/home/kvalem/.local/lib/python3.11/site-packages/numba/core/cpu.py:99: UserWarning: Numba extension module 'awkward.numba' failed to load due to 'ModuleNotFoundError(No module named 'importlib_metadata')'.
  numba.core.entrypoints.init_all()


In [2]:
path = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/011_analysis_paired_remove_xy/pseudobulk/"
resDir_figures = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/figures/LUAD_DE/"
resDir_tables = "/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/"
input_path = f"{path}/paired_adata_clean.h5ad"

In [3]:
adata = sc.read_h5ad(input_path) 

In [4]:
adata

AnnData object with n_obs × n_vars = 466234 × 17811
    obs: 'sample', 'uicc_stage', 'ever_smoker', 'age', 'donor_id', 'origin', 'dataset', 'ann_fine', 'cell_type_predicted', 'doublet_status', 'leiden', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'ann_coarse', 'cell_type_tumor', 'tumor_stage', 'EGFR_mutation', 'TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation', 'KRAS_mutation', 'ROS_mutation', 'origin_fine', 'study', 'platform', 'cell_type_major', 'cell_type_neutro', 'cell_type_neutro_coarse', 'suspension_type', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts',

## There are malignant cells in normal_adjacent samples
Comming from 59 different patients and 14 datasets
I cannot give an explanation for this so i remove this "mislalbeled" cells 

In [5]:
grouped = adata.obs.groupby(['cell_type', 'origin']).size().reset_index(name='cell_count')

In [6]:
grouped

,cell_type,origin,cell_count
0,epithelial cell,normal_adjacent,5532
1,epithelial cell,tumor_primary,17420
2,macrophage,normal_adjacent,82361
3,macrophage,tumor_primary,48086
4,B cell,normal_adjacent,4121
5,B cell,tumor_primary,23779
6,dendritic cell,normal_adjacent,446
7,dendritic cell,tumor_primary,712
8,CD4+ T cell,normal_adjacent,57758
9,CD4+ T cell,tumor_primary,62899


In [7]:
# Create a Boolean mask for the cells that you want to keep
mask = ~((adata.obs['origin'] == 'normal_adjacent') & (adata.obs['cell_type'] == 'malignant cell'))

# Subset the adata object using the mask to exclude the specific cells
adata = adata[mask].copy()

# Now `adata_filtered` contains all cells except those with origin == 'normal_adjacent' and cell_type == 'malignant cell'


In [8]:
grouped = adata.obs.groupby(['cell_type', 'origin']).size().reset_index(name='cell_count')
grouped

,cell_type,origin,cell_count
0,epithelial cell,normal_adjacent,5532
1,epithelial cell,tumor_primary,17420
2,macrophage,normal_adjacent,82361
3,macrophage,tumor_primary,48086
4,B cell,normal_adjacent,4121
5,B cell,tumor_primary,23779
6,dendritic cell,normal_adjacent,446
7,dendritic cell,tumor_primary,712
8,CD4+ T cell,normal_adjacent,57758
9,CD4+ T cell,tumor_primary,62899


In [9]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [10]:
set(adata_normal.obs.cell_type)

{'B cell',
 'CD4+ T cell',
 'CD8+ T cell',
 'dendritic cell',
 'epithelial cell',
 'macrophage',
 'neutrophil',
 'regulatory T cell'}

In [11]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [12]:
set(adata_normal.obs.cell_type)

{'B cell',
 'CD4+ T cell',
 'CD8+ T cell',
 'dendritic cell',
 'epithelial cell',
 'macrophage',
 'neutrophil',
 'regulatory T cell'}

In [13]:
adata_normal = adata[adata.obs["origin"]=="normal_adjacent"]
#adata_normal = adata_normal[~adata_normal.obs["cell_type"].isin(["malignant cell"])]

In [14]:
adata_tumor = adata[adata.obs["origin"]=="tumor_primary"]

In [15]:
adata

AnnData object with n_obs × n_vars = 465474 × 17811
    obs: 'sample', 'uicc_stage', 'ever_smoker', 'age', 'donor_id', 'origin', 'dataset', 'ann_fine', 'cell_type_predicted', 'doublet_status', 'leiden', 'n_genes_by_counts', 'total_counts', 'total_counts_mito', 'pct_counts_mito', 'ann_coarse', 'cell_type_tumor', 'tumor_stage', 'EGFR_mutation', 'TP53_mutation', 'ALK_mutation', 'BRAF_mutation', 'ERBB2_mutation', 'KRAS_mutation', 'ROS_mutation', 'origin_fine', 'study', 'platform', 'cell_type_major', 'cell_type_neutro', 'cell_type_neutro_coarse', 'suspension_type', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'is_highly_variable', 'mito', 'n_cells_by_counts',

In [16]:
adata_male = adata[adata.obs["sex"]=="male"]

In [17]:
adata_female = adata[adata.obs["sex"]=="female"]

## Prepare for pseudobulk

In [18]:
contrasts = [
        dict(var = "origin", condition = "tumor_primary", reference = "normal_adjacent"),
    ]

In [19]:
contrasts

[{'var': 'origin',
  'condition': 'tumor_primary',
  'reference': 'normal_adjacent'}]

In [20]:
contrasts[0]["condition"].replace(" ", "_") + "_vs_" + contrasts[0]["reference"].replace(" ", "_")

'tumor_primary_vs_normal_adjacent'

In [22]:
resDir_tables

'/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/012_LUAD/03012025/tables/LUAD_DE/'

In [22]:
for contrast in contrasts:
    name = contrast["condition"].replace(" ", "_") + "_vs_" + contrast["reference"].replace(" ", "_")
    contrast["name"] = name
    res_dir = Path(resDir_tables, name, "tables")
    os.makedirs(resDir_tables, mode = 0o750, exist_ok = True)
    contrast["res_dir"] = resDir

In [23]:
cell_type_class = "cell_type"

In [24]:
contrast["res_dir"]

'/data/projects/2023/LCBiome/nsclc_gender_atlas_tmp/out/011_analysis_paired_remove_xy/figures/'

In [ ]:
pdata = dc.get_pseudobulk(adata,
                          sample_col='sample',
                          groups_col=cell_type_class,
                          layer='count',
                          mode='sum',
                          min_cells=10,
                          min_counts=1000
                         )

In [ ]:
pdata.X = pdata.X.astype(int)

In [ ]:
pdata.var_names = pdata.var["feature_name"].astype(str)

In [ ]:
pdata_obs_df = pdata.obs

In [ ]:
pdata

In [ ]:
# Group by patient and filter for patients who have samples in both 'tumor' and 'normal_adjacent' conditions
patients_with_both_conditions = pdata_obs_df.groupby('donor_id')['origin'].apply(lambda x: set(x) >= {'tumor_primary', 'normal_adjacent'})

In [ ]:
# Get the patient IDs that meet the condition
patients_with_both_conditions_ids = patients_with_both_conditions[patients_with_both_conditions].index

In [ ]:
pdata_tumor_normal = pdata[pdata.obs['donor_id'].isin(patients_with_both_conditions_ids)]

In [ ]:
cell_type_class

In [ ]:
## Run deseq2 on pseudobulk all cell types
#cell_types = pdata_tumor_normal.obs[cell_type_class].unique()

In [ ]:
contrast["reference"]

In [ ]:
contrast["condition"]

In [ ]:
pdata_tumor_normal.obs['sex'] = pdata_tumor_normal.obs['sex'].astype('category')

In [ ]:
cpus=16

In [ ]:
pdata_tumor_normal

In [ ]:
pdata_tumor_normal.obs["cell_type"]= pdata_tumor_normal.obs["cell_type"].replace(['epithelial cell of lung','multi-ciliated epithelial cell',], 'epithelial cell')
pdata_tumor_normal.obs["cell_type"]= pdata_tumor_normal.obs["cell_type"].replace(['alveolar macrophage'], 'macrophage')
pdata_tumor_normal.obs["cell_type"]= pdata_tumor_normal.obs["cell_type"].replace(['CD4-positive, alpha-beta T cell'], 'CD4+ T cell')
pdata_tumor_normal.obs["cell_type"]= pdata_tumor_normal.obs["cell_type"].replace(['CD8-positive, alpha-beta T cell'], 'CD8+ T cell')

In [ ]:
pdata_tumor_normal.obs["cell_type"].value_counts()

In [ ]:
pdata_tumor_normal

In [ ]:
pdata_only_tumor = pdata_tumor_normal[pdata_tumor_normal.obs["origin"]=="tumor_primary"]

In [ ]:
pdata_only_tumor

In [ ]:
resDir

In [ ]:
cell_types = pdata_only_tumor.obs[cell_type_class].unique()

In [ ]:
cell_types

## Pydeseq tumor vs normal

In [ ]:
import pdb
for contrast in contrasts:
    de_res = {}

    for ct in cell_types:
        print("Working on: " + ct)
        pb_ct = pdata_only_tumor[pdata_only_tumor.obs[cell_type_class] == ct].copy()
        
     
        #pdb.set_trace()
        if len(set(pb_ct.obs["origin"]).intersection([contrast["reference"], contrast["condition"]])) < 1:
             print(
                 "Not running DEseq for: "
                  + ct
                  + " : only present in: "
                  + str(set(pb_ct.obs["sex"]).intersection([contrast["reference"], contrast["condition"]]))
             )
             continue
        #pdb.set_trace()
                     
        dds = DeseqDataSet(
           adata=pb_ct,
           design_factors=[contrast["var"]],
           ref_level=[contrast["var"], 'female'],
           refit_cooks=True,
           n_cpus=cpus,
           )
             
        

        # Compute LFCs
        dds.deseq2()

 

        

        # Extract contrast
        stat_res = DeseqStats(
              dds,
              contrast=[contrast["var"],'male', 'female'],
              #n_cpus=cpus,
          )
   
    

        # Compute Wald test
        stat_res.summary()

        # Shrink LFCs
        coeff = contrast["var"] + "_" + contrast["name"]
        stat_res.lfc_shrink(coeff=coeff)

        # Register cell type results
        de_res[ct] = stat_res.results_df
        de_res[ct]["cell_type"] = ct
        de_res[ct]["feature_name"] = stat_res.results_df.index.values

        de_res[ct].drop(columns=["feature_name"]).to_csv(
                Path(
                    contrast["res_dir"],
                    ct.replace(" ", "_") + "_" + contrast["name"] + "_deseq.tsv",
                ),
                sep="\t",
            )

        # Register results for current contrast
        contrast["de_res"] = de_res
        contrast["de_res_all"] = pd.concat([df.assign(cell_type=ct) for ct, df in de_res.items()])

In [ ]:
resDir

In [ ]:
cell_types = contrast["de_res_all"]["cell_type"].unique()

# Loop through each cell type and save its data to CSV
for cell_type in cell_types:
    # Filter the DataFrame for the current cell type
    ct_all_deg = contrast["de_res_all"][contrast["de_res_all"]["cell_type"] == cell_type]
    
    # Remove spaces from cell_type for the filename
    cell_type_filename = cell_type.replace(" ", "")
    
    # Save the full data for the cell type
    filename_deg = f"{cell_type_filename}_deg.csv"
    filepath_deg = os.path.join(resDir, filename_deg)
    ct_all_deg.to_csv(filepath_deg, index=False)
    print(f"Saved full {cell_type} data to {filepath_deg}")
    
    # Apply additional filtering for significant results
    filtered_df = ct_all_deg[(ct_all_deg['padj'] < 0.1) & (ct_all_deg['log2FoldChange'].abs() > 1)]
    
    # Save the filtered significant data for the cell type
    filename_sig_deg = f"{cell_type_filename}_sig_deg.csv"
    filepath_sig_deg = os.path.join(resDir, filename_sig_deg)
    filtered_df.to_csv(filepath_sig_deg, index=False)
    print(f"Saved significant {cell_type} data to {filepath_sig_deg}")

In [ ]:
pdata_only_tumor.obs.origin.value_counts()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Assuming `contrast` and `dc` are already defined and set up in your environment
cell_types = contrast["de_res_all"]["cell_type"].unique()

# Create a PDF file to save all volcano plots
with PdfPages(f"{resDir_figures}/volcano_plots_all_cell_types.pdf") as pdf:
    for cell_type in cell_types:
        # Filter results for the current cell type
        results_df = contrast["de_res_all"][contrast["de_res_all"]["cell_type"] == cell_type]

        # Generate the volcano plot
        plt.figure(figsize=(8, 4))
        dc.plot_volcano_df(
            results_df,
            x='log2FoldChange',
            y='padj',
            top=20,
            figsize=(8, 4)
        )
        
        # Set title to indicate cell type
        plt.title(f"Volcano Plot for Cell Type: {cell_type}")
        
        # Save the current figure to the PDF
        pdf.savefig()
        plt.close()  # Close the plot to avoid display issues in the next iteration

print("PDF with all volcano plots saved as 'volcano_plots_all_cell_types.pdf'")